In [14]:
import os
import hashlib
import pandas as pd

csv_path = os.path.join(os.curdir, "datasets", "dataset - Privacy-Engineering.csv")
df = pd.read_csv(csv_path)

In [15]:
columns_to_drop = ["ancestry"]
df = df.drop(columns_to_drop, axis=1)
df.head(5)

,id,gender,dob,zipcode,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,female,12/1950,26904,High School,Retired,2,married,2,0.0,Own house,Alzheimer's disease
1,Lillian Cole,female,4/1940,78088,PhD/md,Retired,1,married,1,0.0,Rent flat,multiple sclerosis
2,Lillian Simpson,female,12/1966,51502,Bachelor,Employed,1,married,0,0.1,Rent flat,heart disease
3,Avery Richards,female,4/1943,54080,Bachelor,Retired,1,married,0,0.0,Rent room,endometriosis
4,Sophia Alvarado,female,10/1947,68785,High School,Retired,1,married,0,0.0,Rent flat,gastritis


In [18]:
salt = b"\x91\xff=s\x18?JH\x940\xf1$wN\xa4\xbf\x9f\xcdi\x8c\xa2\xf3\xe3I\xeb\xf4\x1f\xc0y\x94\x84\xa8"

df.id = df.id.apply(lambda x : hashlib.sha512(str(x).encode() + salt).hexdigest())

In [19]:
df.head(5)

,id,gender,dob,zipcode,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease
0,1141b567bad57e77d361c1a63e2b7c6071deda204b2e23...,female,12/1950,26904,High School,Retired,2,married,2,0.0,Own house,Alzheimer's disease
1,97e128377c7d0d92f83a3c2e361446bd8e570c1b57b7fc...,female,4/1940,78088,PhD/md,Retired,1,married,1,0.0,Rent flat,multiple sclerosis
2,2f1d90f629a4ceba4a135fb1f903d62ffc24424a63fcee...,female,12/1966,51502,Bachelor,Employed,1,married,0,0.1,Rent flat,heart disease
3,d26355db08f6123e10f63cd5e3711719133d6fc746da00...,female,4/1943,54080,Bachelor,Retired,1,married,0,0.0,Rent room,endometriosis
4,ad39f73e9baf6a40bc384d54bf6ce6465f20fc58e93790...,female,10/1947,68785,High School,Retired,1,married,0,0.0,Rent flat,gastritis
